
Copyright

Jelen forráskód a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük
az alábbi szerzőt értesíteni.

2019 (c) Csapó Tamás Gábor (csapot kukac tmit pont bme pont hu),
Gyires-Tóth Bálint, Zainkó Csaba


Links:
    [glove] https://nlp.stanford.edu/projects/glove/
    [glove.6B.50d.txt] http://nlp.stanford.edu/data/glove.6B.zip


In [1]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip glove.6B.50d.txt

--2019-11-29 15:14:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  1.99MB/s    in 6m 29s  

2019-11-29 15:20:41 (2.11 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        


In [2]:
#Vektorok beolvasása
import numpy as np
words = set()
vectors = {}
with open('glove.6B.50d.txt', 'r', encoding="utf8", errors='ignore') as f:
  for line in f:
    line = line.strip().split()
    cword = line[0]
    
    try:
      v= np.array(line[1:], dtype=np.float64)
      if v.shape[0] == 50:
          vectors[cword]=v
          words.add(cword)
      else:
          print("Error in line: {}".format(line))
    except:
          print("Error in line: {}".format(line))
print("Number of read words: {}".format(len(words)))


Number of read words: 400000


In [0]:
#Hasonló szavak keresése koszinusz hasonlóság alapján
def find_similarity(word):
    word=word.lower()
    v1=vectors[word]
    top = 5  # top 5 tárolása
    max= np.ones(top)* -200
    mwords= np.chararray(top,itemsize=20,unicode=True)
    mwords[:]= None
    
    #cos similarity
    u = np.sqrt(np.sum(np.square(v1)))
    
    for w in words:  
        if w in word:
           continue      
        vm=vectors[w]
        uv = np.dot(v1,vm)
        v = np.sqrt(np.sum(np.square(vm)))
        cosine_sim=  uv/(u*v)
    
        
        for i in range(top):
          if cosine_sim > max[i]:
              for j in range(top-1,i,-1):
                max[j]=max[j-1]
                mwords[j]=mwords[j-1]
              #print(w)
              mwords[i]=w
              max[i]=cosine_sim
              break
            
        
    return mwords,max



In [0]:
#Hasonlók keresése és kiíratása 
def find_and_print_similarity(w1):
  print("Find similarity: {}".format(w1))
  top5_words,values=find_similarity(w1)

  for i, w in enumerate(top5_words):
     print("{}. {:6.4f} : {}".format(i,values[i],w))

In [5]:
find_and_print_similarity("dollar")

Find similarity: dollar
0. 0.8264 : euro
1. 0.8199 : currency
2. 0.7883 : dollars
3. 0.7817 : greenback
4. 0.7810 : price


In [6]:
find_and_print_similarity("sea")

Find similarity: sea
0. 0.8812 : ocean
1. 0.8739 : waters
2. 0.8261 : seas
3. 0.7974 : coast
4. 0.7801 : mediterranean


In [0]:
#Analógia keresése a vektortérben
def find_analogy(word1, word2, word3):
    word1=word1.lower()
    word2=word2.lower()
    word3=word3.lower()
    
    v1=vectors[word1]
    v2=vectors[word2]
    v3=vectors[word3]
 
    top = 5 
    max= np.ones(top)* -200
    mwords= np.chararray(top,itemsize=20,unicode=True)
    mwords[:]= None
    
    #cos similarity
    v21= v2 - v1
    
    u = np.sqrt(np.sum(np.square(v21)))
    
    for w in words:        
        if w in [word1, word2, word3] :
            continue
       
        vm=vectors[w] -v3
        uv = np.dot(v21,vm)
        v = np.sqrt(np.sum(np.square(vm)))
        cosine_sim=  uv/(u*v)
    
        
        for i in range(top):
          if cosine_sim > max[i]:
              for j in range(top-1,i,-1):
                max[j]=max[j-1]
                mwords[j]=mwords[j-1]
              #print(w)
              mwords[i]=w
              max[i]=cosine_sim
              break
            
        
    return mwords,max



In [0]:
#Analógia keresése és a top 5 kiíratása
def find_and_print_analogy(w1,w2,w3):
  print("Find analogy: {} - {} + {} =".format(w1,w2,w3))
  top5_words,values=find_analogy(w1,w2,w3)

  for i, w in enumerate(top5_words):
     print("{}. {:6.4f} : {}".format(i,values[i],w))

In [9]:
find_and_print_analogy('woman', 'queen', 'man')

Find analogy: woman - queen + man =
0. 0.8397 : king
1. 0.8330 : coronation
2. 0.7883 : prince
3. 0.7693 : crown
4. 0.7607 : vi


In [10]:
find_and_print_analogy('France', 'Paris', 'Germany')

Find analogy: France - Paris + Germany =
0. 0.7957 : berlin
1. 0.7081 : aviv
2. 0.6967 : tel
3. 0.6801 : soho
4. 0.6776 : stockholm


In [11]:
find_and_print_analogy('usa', 'soccer', 'England')

Find analogy: usa - soccer + England =
0. 0.3973 : premiership
1. 0.3755 : football
2. 0.3536 : scolari
3. 0.3407 : squad
4. 0.3243 : rugby


In [12]:
find_and_print_analogy('Europeans', 'euro', 'americans')

Find analogy: Europeans - euro + americans =
0. 0.8170 : dollar
1. 0.7368 : deficit
2. 0.7079 : net
3. 0.6972 : drop
4. 0.6963 : quarter


In [13]:
find_and_print_analogy('car', 'road', 'boat')

Find analogy: car - road + boat =
0. 0.6125 : river
1. 0.5757 : estuary
2. 0.5592 : transfrontier
3. 0.5576 : parkway
4. 0.5543 : peninsula
